In [28]:
import numpy as np
import statsmodels as sm
from cmdstanpy import cmdstan_path, CmdStanModel
import math
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

1. A randomized experiment is performed within a survey. 1000 people are contacted. Half the people contacted are promised a $5 incentive to participate, and half are not promised an incentive. The result is a 50% response rate among the treated group and 40% response rate among the control group.

 a. Give an estimate and standard error of the average treatment effect.

 b. Give code to fit a logistic regression of response on the treatment indicator. Give the complete code, including assigning the data, setting up the variables, etc. It is not enough to simply give the one line of code for running the logistic regression.

### Estimates

#### Stan

In [ ]:
'''
parameters {
  real alpha;  //  treatment
  real beta;  // control
}
model {
  250 ~ binomial_logit(500, alpha);
  200 ~ binomial_logit(500, beta);
}
generated quantities {
  real delta = alpha - beta;
}    
'''
#via Bob Carpenter in Gelman blog comments

In [80]:
bino_model = CmdStanModel(stan_file='exercise1.stan')
bino_fit = bino_model.sample('./output')

INFO:cmdstanpy:compiling stan program, exe file: /Users/spetulla/Workspace/Sandbox/ds-practice/Regression/exercise1
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /Users/spetulla/Workspace/Sandbox/ds-practice/Regression/exercise1
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


In [81]:
bino_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-684.061000,0.022687,0.984280,-685.992000,-683.762000,-683.128000,1882.35,29479.0,1.00040
alpha,-0.000021,0.001601,0.090096,-0.150502,0.002382,0.146618,3168.07,49614.2,1.00038
beta,-0.406683,0.001502,0.089006,-0.551859,-0.405274,-0.260359,3513.51,55024.2,1.00020
delta,0.406662,0.002235,0.128011,0.197845,0.406044,0.610964,3279.79,51363.9,1.00097


In [184]:
print(f'Stan estimate: {inv_logit(0.002382) - inv_logit(-0.405274)}')

Stan estimate: 0.10054963289605962


#### Analytic estimate

In [171]:
a = 0.5 - 0.4
# se = sqrt(σ1/n1 + σ2/n2)
σ1 = 0.5*0.5
σ2 = 0.6*0.4
n1,n2 = 500,500
se = math.sqrt((σ1/n1) + (σ2/n2))
print(f'estimate: {a}, se: {se}')

estimate: 0.09999999999999998, se: 0.03130495168499706


#### Logistic regression and interpretation

In [151]:
lr = LogisticRegression()
# 1 = treatment
# 0 = control
t = [1] * 500 + [0] * 500
y = [1] * 250 + [0] * 250 + [1] * 200 + [0] * 300

lr.fit(np.array(t).reshape(-1,1), np.array(y).reshape(-1,1))
print('c,t',*lr.coef_[0], *lr.intercept_)

c,t 0.3989500027010817 -0.4021416132970716


In [146]:
#odds ratio of control to treatment
print(math.exp(lr.intercept_))
#odds ratio of treatment to control
print(math.exp(lr.coef_))
# for se, use statsmodel

0.6688860158289345
1.4902591078146699


In [145]:
# matches analytical odds ratios
print((200/300) / (250/250))
print((250/250) / (200/300))

0.6666666666666666
1.5
